In [ ]:
import numpy as np
import glob
import os
from tqdm import tqdm
import re
import gc
import my_utils as myu  # Ensure 'myu' is defined with necessary functions

ending_iterations = 200000

def check_files_for_folder(folder_path, required_early=10, required_later=20, min_later_number=160001, last_later_threshold=ending_iterations):
    """Check the number of early and later files in the folder and ensure the last later file's number > 200,000."""
    all_vtk_files = glob.glob(os.path.join(folder_path, 'outputs/*.vtk'))  # Get all VTK files in the folder
    file_details = []
    messages = []  # List to store messages instead of printing them directly

    for file in all_vtk_files:
        try:
            num = int(file.split('_')[-1].split('.')[0])  # Extract frame number from file name
            file_details.append((file, num))
        except ValueError:
            messages.append(f"Warning: Unable to parse file number from '{file}'. Skipping this file.")

    file_details_sorted = sorted(file_details, key=lambda x: x[1])  # Sort files based on frame number

    # Separate early files (<= 160000) and later files (>= min_later_number)
    early_files = [(file, num) for file, num in file_details_sorted if num <= 160000]
    later_files = [(file, num) for file, num in file_details_sorted if num >= min_later_number]

    # Select a limited number of early files (maximum 10)
    if len(early_files) >= required_early:
        selected_indices_early = np.linspace(0, len(early_files) - 1, required_early, dtype=int)  # Select required_early evenly spaced files
        early_selected = [early_files[i][0] for i in selected_indices_early]  # Select only file paths
    else:
        messages.append(f"Warning: Not enough early files in '{folder_path}'. Required: {required_early}, Found: {len(early_files)}.")
        early_selected = [file for file, num in early_files]  # Select all available file paths

    # Select a limited number of later files (maximum 20)
    if len(later_files) >= required_later:
        selected_indices_later = np.linspace(0, len(later_files) - 1, required_later, dtype=int)  # Select required_later evenly spaced files
        later_selected = [later_files[i][0] for i in selected_indices_later]  # Select only file paths
    elif len(later_files) > 0:
        messages.append(f"Warning: Not enough later files in '{folder_path}'. Required: {required_later}, Found: {len(later_files)}.")
        later_selected = [file for file, num in later_files]  # Select all available file paths
    else:
        messages.append(f"Error: No files found with numbers >= {min_later_number} in '{folder_path}'. Skipping this folder.")
        return None, None, None, messages  # Skip the folder if no valid files are found

    # Handle the last file in later_files and check if it's corrupted or has an issue
    last_file, last_number = None, None
    valid_last_file = False

    for idx in reversed(range(len(later_files))):  # Loop backwards through the later_files list
        try:
            # Try to get the file and its number
            last_file = later_files[idx][0]
            last_number = later_files[idx][1]

            # Check if the file has a number > 200,000
            if last_number > last_later_threshold:
                valid_last_file = True
                messages.append(f"The file '{last_file}' has a valid number > {last_later_threshold}: {last_number}.")
                break  # If the file is valid, exit the loop
            else:
                messages.append(f"Warning: The file '{last_file}' has a number <= {last_later_threshold}: {last_number}. Trying previous file.")

        except (IndexError, ValueError, OSError) as e:
            # Handle the case where the file is corrupted or unreadable
            messages.append(f"Error: Failed to process file '{last_file}' in '{folder_path}'. Error: {e}. Trying previous file.")
    
    # If no valid last file was found, skip the folder
    if not valid_last_file:
        messages.append(f"Error: No valid file with a number greater than {last_later_threshold} was found in '{folder_path}'. Skipping this folder.")
        return None, None, None, messages

    return early_selected, later_selected, last_number, messages

def find_global_bounds(subfolders):
    """Calculate global bounds based on the 30th VTK file in the later files across all subfolders."""
    global_min_x, global_min_y = float('inf'), float('inf')  # Initialize min values to infinity
    global_max_x, global_max_y = float('-inf'), float('-inf')  # Initialize max values to negative infinity

    folder_file_info = {}  # Dictionary to store file information for each folder
    problematic_folders = []  # List to collect problematic folders and issues
    all_messages = []  # List to collect all messages instead of printing

    # Loop through each subfolder to find the global bounds and gather file information
    for subfolder in tqdm(subfolders, desc="Calculating global bounds"):
        early_files, later_files, last_later_number, messages = check_files_for_folder(subfolder)
        all_messages.extend(messages)  # Collect messages from check_files_for_folder

        # Skip folder if the last_later_number is smaller than ending_iterations
        if last_later_number is not None and last_later_number < ending_iterations:
            problematic_folders.append(f"Skipping folder '{subfolder}' as last_later_number is less than ending_iterations: {last_later_number}.")
            continue

        if later_files is None:  # Skip the folder if no valid files are found
            problematic_folders.append(f"Skipping folder '{subfolder}' due to no valid later files.")
            continue

        folder_file_info[subfolder] = (early_files, later_files, last_later_number)

        # Loop through later_files in reverse to find a valid file for bounds calculation
        valid_file_found = False
        for idx in reversed(range(len(later_files))):
            last_file = later_files[idx]
            try:
                data = myu.read_mesh_cellCon_exception(last_file, 1, 0)  # Read the mesh and field data
                if data:
                    points = data[0]  # Extract points data
                    # Validate points to ensure they are numeric and finite
                    if np.isfinite(points).all():  # Ensure there are no `inf` or `nan` values in points
                        min_x, min_y = np.min(points, axis=0)  # Get min x and y values
                        max_x, max_y = np.max(points, axis=0)  # Get max x and y values
                        # Update the global bounds based on the current file
                        global_min_x = min(global_min_x, min_x)
                        global_min_y = min(global_min_y, min_y)
                        global_max_x = max(global_max_x, max_x)
                        global_max_y = max(global_max_y, max_y)
                        valid_file_found = True  # A valid file was found
                        break  # Exit loop after finding a valid file
                    else:
                        problematic_folders.append(f"Error: Invalid (non-finite) points in file '{last_file}'. Skipping.")
                else:
                    problematic_folders.append(f"Error reading mesh data for file '{last_file}' in folder '{subfolder}'. Skipping.")

            except (IndexError, ValueError, OSError) as e:
                # Handle the case where the last file is corrupted or unreadable
                problematic_folders.append(f"Error processing file '{last_file}' in folder '{subfolder}'. Error: {e}. Trying previous file.")

        # If no valid file was found, skip the folder
        if not valid_file_found:
            problematic_folders.append(f"Error: No valid file found for bounds calculation in '{subfolder}'. Skipping this folder.")
            continue

    # Print out problematic folders at the end
    if problematic_folders:
        all_messages.append("\nProblematic folders encountered:")
        all_messages.extend(problematic_folders)

    # Print all messages at once
    if all_messages:
        print("\n".join(all_messages))

    # Check if bounds are valid (ensure no inf bounds)
    if not np.isfinite([global_min_x, global_min_y, global_max_x, global_max_y]).all():
        raise ValueError("Error: Invalid global bounds detected. Check the input data.")

    # Return the global bounds and the collected file information
    return global_min_x, global_min_y, global_max_x, global_max_y, folder_file_info

def interpolate_data_for_folder(folder_path, global_bounds, file_info, grid_size=1.0):
    """Interpolate selected VTK files within a folder using specified global bounds and file information."""
    global_min_x, global_min_y, global_max_x, global_max_y = global_bounds
    x_coords = np.arange(global_min_x, global_max_x + grid_size, grid_size)
    y_coords = np.arange(global_min_y, global_max_y + grid_size, grid_size)
    grid_x, grid_y = np.meshgrid(x_coords, y_coords)
    grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T.astype(np.float32)  # Use float32 to save memory

    early_files, later_files, last_later_number = file_info  # Use the pre-collected file information

    # Skip folder if the last_later_number is smaller than ending_iterations
    if last_later_number is not None and last_later_number < ending_iterations:
        print(f"Skipping folder '{folder_path}' as last_later_number is less than ending_iterations: {last_later_number}.")
        return None, None, None

    selected_files = early_files + later_files

    if not selected_files:
        print(f"Error: No valid files selected in '{folder_path}'. Skipping this folder.")
        return None, None, None

    interpolated_data = []
    last_valid_step_data = None

    folder_name = os.path.basename(folder_path)  # Get the folder name for display
    desc = f'Interpolating in {folder_name}'  # Description for progress bar

    # Process each selected file
    with tqdm(total=len(selected_files), desc=desc, unit='file') as progress:
        for filename in selected_files:
            data = myu.read_mesh_cellCon_exception(filename, 1, 0)  # Read the mesh and field data
            if data:
                points, fields, _, _ = data  # Extract points and scalar fields
                interpolated_step_data = {}
                for field_name, values in fields.items():
                    try:
                        # Interpolate each field using the grid points
                        interpolated_values = myu.interpolate_features_cKD(points, {field_name: values}, grid_points, k=3)
                        interpolated_matrix = interpolated_values[field_name].reshape(len(y_coords), len(x_coords))
                        interpolated_step_data[field_name] = interpolated_matrix  # Store the interpolated matrix
                    except Exception as e:
                        print(f"Warning: Interpolation failed for field '{field_name}' in file '{filename}': {e}")
                        interpolated_step_data[field_name] = np.full((len(y_coords), len(x_coords)), np.nan)  # Assign NaNs on failure
                interpolated_data.append(interpolated_step_data)  # Append interpolated data for this file
                last_valid_step_data = interpolated_step_data  # Update last valid data

                # Release memory after each file is processed
                del points, fields, interpolated_step_data
                gc.collect()

            else:
                if last_valid_step_data is not None:
                    print(f"Info: Using previous valid data for file '{filename}'.")
                    interpolated_data.append(last_valid_step_data.copy())  # Use the last valid data
                else:
                    print(f"Warning: No previous valid data to duplicate for file '{filename}'. Assigning NaNs.")
                    # Assign NaNs for all fields if no previous data exists
                    duplicated_step_data = {field: np.full((len(y_coords), len(x_coords)), np.nan) for field in myu.REQUIRED_FIELDS}
                    interpolated_data.append(duplicated_step_data)

            progress.update(1)  # Update progress bar

    return interpolated_data, grid_x, grid_y  # Return the interpolated data and grid

def process_all_cases(base_path):
    """Process all cases (subfolders) in the specified base path."""
    subfolders = glob.glob(os.path.join(base_path, 'io2D_ND*'))  # Get all subfolders matching the pattern

    # Sort subfolders based on the numeric part of the folder name (e.g., ND001, ND002, etc.)
    subfolders.sort(key=lambda x: int(re.search(r'ND(\d+)', os.path.basename(x)).group(1)))
    if not subfolders:
        print(f"Error: No subfolders found in '{base_path}'.")
        return {}

    # Unpack all four global bounds (min_x, min_y, max_x, max_y) and file info
    global_min_x, global_min_y, global_max_x, global_max_y, folder_file_info = find_global_bounds(subfolders)
    
    global_bounds = (global_min_x, global_min_y, global_max_x, global_max_y)  # Pack global bounds into a tuple
    print(f"Global bounds determined: {global_bounds}")
    
    cases_data = {}  # Dictionary to store the interpolated data for each case

    # Process each subfolder
    for subfolder in subfolders:
        file_info = folder_file_info.get(subfolder)  # Retrieve file info for this folder
        if file_info:  # Only proceed if valid file info is available
            interpolated_data, grid_x, grid_y = interpolate_data_for_folder(subfolder, global_bounds, file_info)
            if interpolated_data is not None:
                case_name = os.path.basename(subfolder)  # Get the case name (subfolder name)
                cases_data[case_name] = (interpolated_data, grid_x, grid_y)  # Store the interpolated data
            else:
                print(f"Warning: No data interpolated for folder '{subfolder}'.")
        else:
            print(f"Skipping folder '{subfolder}' due to insufficient files.")

    return cases_data  # Return the processed data for all cases

# root_folder_path = "/mnt/e/NeuronGrowth_2024/TTNG_05052024"
# root_folder_path = "/mnt/e/GenNDD_data"
root_folder_path = "/ocean/projects/eng170006p/ussqww/GenNDD_data"

all_cases_data = process_all_cases(root_folder_path)

In [ ]:
import numpy as np
from scipy.ndimage import zoom

def downsample_all_cases_data(all_cases_data, target_size=(160, 280)):
    """
    Downsamples each dataset from (241, 241) to (160, 280) for each image and the grid in all_cases_data.

    Args:
    all_cases_data (dict): Dictionary with keys as case identifiers and values as tuples of
                           (interpolated_data, grid_x, grid_y) where interpolated_data is a list of dictionaries
                           of shape (#time, #channel, 241, 241).
    target_size (tuple): Target downsample size, default (160, 280).

    Returns:
    dict: A dictionary with the same keys and downsampled data as values. Each case will have:
          (downsampled_interpolated_data, downsampled_grid_x, downsampled_grid_y).
    """
    downsampled_data = {}

    for key, (interpolated_data, grid_x, grid_y) in all_cases_data.items():
        time_steps = len(interpolated_data)
        channels = interpolated_data[0].keys()  # Assuming all time steps have the same channels
        downsampled_tensor = []

        # Downsample the grid_x and grid_y to (160, 280)
        zoom_factor_grid_x = target_size[0] / grid_x.shape[0]
        zoom_factor_grid_y = target_size[1] / grid_y.shape[1]
        downsampled_grid_x = zoom(grid_x, zoom_factor_grid_x, order=1)  # Bilinear interpolation
        downsampled_grid_y = zoom(grid_y, zoom_factor_grid_y, order=1)

        # Downsample the interpolated_data (each time step and channel)
        for time_step in interpolated_data:
            downsampled_channels = {}
            for channel in channels:
                image = time_step[channel]  # Get the image for this channel
                # Calculate the zoom factor for the image
                zoom_factor_image_x = target_size[0] / image.shape[0]
                zoom_factor_image_y = target_size[1] / image.shape[1]
                # Downsample the image
                downsampled_image = zoom(image, (zoom_factor_image_x, zoom_factor_image_y), order=1)  # Bilinear interpolation (order=1)
                downsampled_channels[channel] = downsampled_image
            downsampled_tensor.append(downsampled_channels)

        # Store the downsampled data for this case
        downsampled_data[key] = (downsampled_tensor, downsampled_grid_x, downsampled_grid_y)

    return downsampled_data

# Example usage
# Assuming `all_cases_data` is your loaded dataset
all_cases_data = downsample_all_cases_data(all_cases_data, target_size=(160, 280))

In [ ]:
def check_consistent_dimensions_all_channels(all_cases_data):
    """
    Check if the dimensions of all channels are consistent across all cases in the dataset.

    Args:
        all_cases_data (dict): Dictionary containing all cases.

    Returns:
        bool: True if dimensions are consistent for all channels, False otherwise.
    """

    # Initialize a variable to store reference shapes for all channels
    reference_shapes = None

    for case, case_data in all_cases_data.items():
        try:
            current_shapes = {channel: case_data[0][0][channel].shape for channel in case_data[0][0]}  # Get shapes for all channels
            
            # Set the first case's shapes as the reference
            if reference_shapes is None:
                reference_shapes = current_shapes
            # Compare the current shapes with the reference shapes
            else:
                for channel, current_shape in current_shapes.items():
                    if channel in reference_shapes:
                        if current_shape != reference_shapes[channel]:
                            print(f"Warning: Shape of '{channel}' in {case} is inconsistent. Expected {reference_shapes[channel]}, but got {current_shape}.")
                            return False
                    else:
                        print(f"Warning: Channel '{channel}' found in {case} but missing in the reference case.")
                        return False
        except KeyError:
            print(f"Warning: Some channels are missing in {case}.")
            return False

    print(f"All cases have consistent dimensions for all channels.")
    return True

def print_all_cases_data_dimensions(all_cases_data):
    """
    Prints the dimensions of the data stored in all_cases_data.

    Args:
        all_cases_data (dict): Dictionary containing data for all cases. 
                               Each case is a tuple with (interpolated_data, grid_x, grid_y).

    Returns:
        None
    """
    for case_name, (interpolated_data, grid_x, grid_y) in all_cases_data.items():
        print(f"Case: {case_name}")
        # Print dimensions of interpolated_data
        if isinstance(interpolated_data, list):
            print(f"  Interpolated Data: {len(interpolated_data)} time steps")
            if len(interpolated_data) > 0 and isinstance(interpolated_data[0], dict):
                for channel, data_array in interpolated_data[0].items():
                    print(f"    Channel '{channel}': {data_array.shape}")
        else:
            print(f"  Interpolated Data: {interpolated_data.shape}")

        # Print dimensions of grid_x and grid_y
        print(f"  Grid X: {grid_x.shape}")
        print(f"  Grid Y: {grid_y.shape}")
        print("-" * 50)

print_all_cases_data_dimensions(all_cases_data)
consistent = check_consistent_dimensions_all_channels(all_cases_data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

def plot_interpolated_time_steps(time_step_data, channel, split_at=5, columns_per_row=3):
    """
    Plots the specified channel from each time step's interpolated data, splitting the plots into two figures.

    Args:
        time_step_data (list of dicts): Each dict contains channel names as keys and 2D arrays of data as values.
        channel (str): Name of the channel to plot.
        split_at (int): Index at which to split the plots into a second figure.
        columns_per_row (int): Number of columns per row in the plot.

    Returns:
        None: Displays the plots.
    """
    
    num_time_steps = len(time_step_data)
    first_batch = min(num_time_steps, split_at)
    second_batch = num_time_steps - first_batch

    # Function to create plots for a given range of time steps
    def plot_batch(start_index, end_index, title):
        num_plots = end_index - start_index
        rows = (num_plots + columns_per_row - 1) // columns_per_row  # Calculate rows needed
        fig, axes = plt.subplots(rows, columns_per_row, figsize=(5 * columns_per_row, 5 * rows))
        if not isinstance(axes, np.ndarray):
            axes = np.array([axes])  # Handle the case where there is only one subplot
        axes = axes.flatten()

        for i in range(num_plots):
            ax = axes[i]
            data_index = start_index + i
            if channel in time_step_data[data_index]:
                c = ax.imshow(time_step_data[data_index][channel], cmap='viridis', aspect='auto')
                fig.colorbar(c, ax=ax)
                ax.set_title(f'Time Step: {data_index + 1}')
                ax.set_xlabel('X Coordinate')
                ax.set_ylabel('Y Coordinate')
            else:
                ax.set_title(f'Channel {channel} not found')
                ax.set_xlabel('X Coordinate')
                ax.set_ylabel('Y Coordinate')
                print(f"Warning: Channel '{channel}' not found in time step {data_index + 1}.")

        for j in range(i + 1, len(axes)):  # Hide unused axes
            axes[j].axis('off')

        plt.suptitle(title)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    # Plot the first batch of time steps
    plot_batch(0, first_batch, "First Batch of Time Steps")

    # Plot the second batch of time steps if there are any
    if second_batch > 0:
        plot_batch(first_batch, num_time_steps, "Second Batch of Time Steps")

# Randomly select a case key from all_cases_data
case_key = random.choice(list(all_cases_data.keys()))  # Randomly pick a case key
channel_to_plot = 'phi'  # Example channel to plot

# Now use the variable to access data and call the plotting function
plot_interpolated_time_steps(all_cases_data[case_key][0], channel_to_plot, columns_per_row=5)


In [ ]:
import numpy as np
import random

def select_and_structure_data(all_cases_data, num_X_steps=10, num_Y_steps=20, train_frac=0.7, val_frac=0.15, channels=None):
    """
    Structured data extraction for ML training from multidimensional time-series data.

    Args:
        all_cases_data (dict): The main dataset containing all cases.
        num_X_steps (int): Number of time steps to use for X.
        num_Y_steps (int): Number of time steps to use for Y.
        train_frac (float): Fraction of data to be used for training.
        val_frac (float): Fraction of data to be used for validation.
        channels (list): List of channel names to include in the output arrays.

    Returns:
        dict: Dataset containing structured data for training, validation, and testing.
    """

    # Extract keys for all cases
    case_keys = list(all_cases_data.keys())
    random.shuffle(case_keys)

    # Determine split indices
    num_train = int(len(case_keys) * train_frac)
    num_val = int(len(case_keys) * val_frac)
    
    train_keys = case_keys[:num_train]
    val_keys = case_keys[num_train:num_train + num_val]
    test_keys = case_keys[num_train + num_val:]

    # Define function to extract data for X and Y
    def extract_data(cases, num_X_steps, num_Y_steps, channels):
        X, Y = [], []
        for case in cases:
            case_data = all_cases_data[case][0]  # Assuming the data structure is (list of dicts with channel data, grid_x, grid_y)
            # print(case_data)
            print(case)
            case_data[0]['phi']
            if len(case_data) >= num_X_steps + num_Y_steps:
                x_segments = np.stack([np.array([time_step[channel] for time_step in case_data[:num_X_steps]]) for channel in channels if channel in case_data[0]], axis=1)
                y_segments = np.stack([np.array([time_step[channel] for time_step in case_data[num_X_steps:num_X_steps + num_Y_steps]]) for channel in channels if channel in case_data[0]], axis=1)

                X.append(x_segments)
                Y.append(y_segments)

        return np.array(X), np.array(Y)

    # Extract data for each dataset
    X_train, Y_train = extract_data(train_keys, num_X_steps, num_Y_steps, channels)
    X_val, Y_val = extract_data(val_keys, num_X_steps, num_Y_steps, channels)
    X_test, Y_test = extract_data(test_keys, num_X_steps, num_Y_steps, channels)

    return {
        'X_train': X_train, 'Y_train': Y_train,
        'X_val': X_val, 'Y_val': Y_val,
        'X_test': X_test, 'Y_test': Y_test
    }

# Example usage: specify which channels to include
# channels_to_include = ['phi', 'synaptogenesis', 'tubulin', 'tips', 'theta']  # Add your specific channels here
channels_to_include = ['phi']  # Add your specific channels here
dataset = select_and_structure_data(all_cases_data, channels=channels_to_include)

# Display the shapes of the datasets
for key in dataset:
    print(f"{key} shape: {dataset[key].shape if dataset[key].size else 'Empty'}")

In [ ]:
import pickle

# Save the dataset to a .pkl file using pickle
with open('./reformatedNDDs/dataset_16k_20k.pkl', 'wb') as f:
    pickle.dump(dataset, f)